### Import Required Package

In [93]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [94]:
import tidal as td

### Initialize Plumber

In [95]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [96]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb/spearman'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

/tmp/ipykernel_23154/2042523340.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_23154/2042523340.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_23154/2042523340.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_d

In [97]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [98]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [99]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_23154/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2022-06-01  119.2856  119.9775  119.0089  119.1473   6730117
           2022-06-02  118.5015  118.5015  117.7173  117.9018   4721923
           2022-06-06  118.3170  119.0089  117.5789  118.7321   4292361
           2022-06-07  118.0402  118.0402  117.0254  117.5328   5160242
           2022-06-08  118.5476  119.1011  118.2247  118.8244   4110313
...                         ...       ...       ...       ...       ...
           2024-09-24  183.2000  185.0000  182.1000  185.0000  10476465
           2024-09-25  187.7000  188.0000  187.3500  187.7500  12464313
           2024-09-26  189.6000  190.0000  188.7500  189.3000  17369543
           2024-09-27  190.2000  191.2500  188.6000  188.9000  11999510
           2024-09-30  186.5500  187.2000  183.9500  183.9500  14461065

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [100]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(15, 3, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

[2024/11/28 14:15:24] root INFO Tidal client version: 1.1.41
[2024/11/28 14:15:24] root ERROR SocketClient tcp://10.136.15.5:6666: No module named 'tidal'
[2024/11/28 14:15:24] root INFO Tidal server version: None


### Stock Config

In [101]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [102]:
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [103]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [104]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [105]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [02:05<00:00,  4.37it/s, cash=6.91e+5, pnl=6.08e+5, position_cost=6.18e+7, value=6.31e+7] 

2024-08-23 - 警告：強制以開盤價 136.5 賣出 8279，因為當天交易量為0
2024-08-27 - 警告：強制以開盤價 139.5 賣出 8279，因為當天交易量為0


### Metric - AccountInfo

In [106]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-07-01  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-04  2.829782e+06  1.545497e+04   4.715476e+07  1.243792e+05   
2022-07-05  4.632924e+05  5.065070e+04   4.948310e+07  3.137391e+05   
2022-07-06  4.736878e+05  8.619253e+04   4.972900e+07  3.695647e+05   
2022-07-07  5.537176e+05  1.232479e+05   4.939350e+07  3.922550e+05   
...                  ...           ...            ...           ...   
2024-09-24  5.236193e+05  1.504472e+07   6.165311e+07  1.356949e+07   
2024-09-25  8.362339e+05  1.508755e+07   6.130531e+07  1.365379e+07   
2024-09-26  6.219499e+05  1.513153e+07   6.160130e+07  1.380819e+07   
2024-09-27  8.412410e+05  1.517671e+07   6.168329e+07  1.367598e+07   
2024-09-30  6.913378e+05  1.522053e+07   6.179543e+07  1.373235e+07   

                     pnl         value  max_drawdown      turnover  
datetim

### Metric - PositionInfo

In [107]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity      price   commission   slip_cost  \
instrument datetime                                                   
1210       2023-07-05   83000.0  49.775051  1354.043167    493.9662   
           2023-07-06   83000.0  49.775051  1354.043167    493.9662   
           2023-07-07   83000.0  49.775051  1354.043167    493.9662   
           2023-07-10   83000.0  49.775051  1354.043167    493.9662   
           2023-07-11   83000.0  49.775051  1354.043167    493.9662   
...                         ...        ...          ...         ...   
9945       2024-09-20   81000.0  47.795400  1268.860330  27977.4000   
           2024-09-25   81000.0  47.795400  1268.860330  36077.4000   
           2024-09-26   81000.0  47.795400  1268.860330  36077.4000   
           2024-09-27   81000.0  47.795400  1268.860330  36077.4000   
           2024-09-30   81000.0  47.795400  1268.860330  36077.4000   

                               pnl  now_price  
instrument datetime         

In [108]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [109]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime                
1101       2022-07-01  4.640564e-06
           2022-07-04  1.653318e-06
           2022-07-05  4.568053e-06
           2022-07-06  5.427003e-07
           2022-07-07  5.575453e-06
...                             ...
9958       2024-09-24  6.189310e-06
           2024-09-25  5.000523e-07
           2024-09-26  2.548680e-06
           2024-09-27  2.142561e-06
           2024-09-30  1.332919e-06

[332693 rows x 1 columns]

### Strategy Lake Submit

In [110]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

### Trade Report

In [111]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6139,2.0,0.0,2.0,0.0,1.000000,5.534185e+06,0.000000e+00,4.670926e+04,5.534185e+06
3617,15.0,19.0,34.0,0.0,0.441176,5.416745e+06,-1.575911e+06,4.458556e+05,3.840834e+06
2392,1.0,0.0,1.0,0.0,1.000000,2.420079e+06,0.000000e+00,2.210516e+04,2.420079e+06
3338,1.0,0.0,1.0,0.0,1.000000,1.517023e+06,0.000000e+00,1.917484e+04,1.517023e+06
6585,1.0,1.0,2.0,0.0,0.500000,1.718951e+06,-2.551232e+05,3.140924e+04,1.463828e+06
...,...,...,...,...,...,...,...,...,...
3289,6.0,8.0,14.0,1.0,0.428571,7.185922e+05,-1.803319e+06,3.961009e+06,-1.084727e+06
6577,1.0,10.0,11.0,0.0,0.090909,1.780180e+03,-1.115677e+06,1.522043e+05,-1.113897e+06
8420,4.0,7.0,11.0,0.0,0.363636,5.701063e+05,-1.793077e+06,1.333207e+05,-1.222971e+06


### Traded instruments

In [112]:
tidal.account.trades.keys()

dict_keys(['6747', '8279', '6531', '8431', '3669', '1785', '8433', '8027', '8091', '7556', '6667', '3551', '2640', '5230', '1580', '8928', '6245', '3617', '1531', '4549', '1817', '4971', '4129', '5284', '1570', '4175', '6799', '4506', '6612', '5222', '6533', '6569', '6561', '6803', '6284', '9917', '2812', '6180', '3567', '4541', '5903', '8420', '3027', '3006', '5371', '5439', '8349', '4945', '2834', '5220', '3546', '8215', '8942', '3025', '2404', '3260', '6523', '6664', '5443', '6206', '8210', '6591', '5607', '6470', '6811', '3594', '6570', '3029', '6139', '8049', '1210', '3038', '1304', '4557', '9938', '2755', '6670', '8938', '6788', '3363', '6691', '4961', '4137', '2637', '2451', '6538', '6147', '4138', '2916', '3532', '2453', '6577', '6776', '5511', '8081', '3707', '3356', '8048', '5508', '6176', '6187', '4938', '8112', '2630', '3694', '3227', '5236', '5287', '6585', '6469', '3213', '1522', '3088', '2392', '3338', '4933', '3622', '6189', '6605', '9927', '3712', '6170', '6290', '3357

### Trading History

In [113]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [114]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [115]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


[2024/11/28 14:17:32] werkzeug INFO WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:39215
 * Running on http://10.136.1.151:39215
[2024/11/28 14:17:32] werkzeug INFO Press CTRL+C to quit


In [ ]:
tidal.account.position_history

In [ ]:
Portfolio_info = tidal.metrics["Portfolio"].report
print(Portfolio_info)

In [ ]:
Portfolio_info.to_csv('./Portfolio_info.csv')